In [1]:
import pandas as pd
import requests
import time
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import numpy as np

In [ ]:
# groq_api_key = api_key

In [2]:

job_descriptions_df = pd.read_parquet("hf://datasets/lang-uk/recruitment-dataset-job-descriptions-english/data/train-00000-of-00001.parquet")
candidate_profiles_df = pd.read_parquet("hf://datasets/lang-uk/recruitment-dataset-candidate-profiles-english/data/train-00000-of-00001.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
job_descriptions_df.head()

,Position,Long Description,Company Name,Exp Years,Primary Keyword,English Level,Published,Long Description_lang,id,__index_level_0__
0,10 + Blockchain Nodes / Masternodes to set up,*Requirements*\r\n\r\nWe're looking for a long...,MyCointainer,2y,Sysadmin,intermediate,2020-10-01T00:00:00+03:00,en,c0ca96e7-85df-50df-a64e-d934cd02a170,27461
1,10 .NET Developers (Middle and Senior level),"Greetings! My name is Maria, I am in urgent ne...",TechScout.tech,2y,.NET,intermediate,2022-03-01T00:00:00+02:00,en,64f4b7ea-36e4-5bdd-a8b1-185f32f7dc7f,27462
2,"10X Engineer (co-founder, #4 employee, USD 11-...",**Product**\r\nThe product is a live video cha...,Innoteka,5y,JavaScript,fluent,2021-07-01T00:00:00+03:00,en,b9a1303e-dd0c-5ed1-8f62-be2bc4c7da4f,27463
3,16 - Amazon Brand Manager,"Currently, TCM expanding its activities to Ukr...",FirstFive,2y,Marketing,upper,2022-01-01T00:00:00+02:00,en,99cb3f4a-9b4b-53d9-9a3b-bab2c22da346,27464
4,16 - Amazon Brand Manager,"Hello,\r\nWe, MIMIRB2B, are an outstaff compan...",MimirB2B,1y,Marketing,upper,2021-12-01T00:00:00+02:00,en,bc1419f7-28e2-582b-8d53-22e28b2f0210,27465


In [ ]:
candidate_profiles_df.head()

,Position,Moreinfo,Looking For,Highlights,Primary Keyword,English Level,Experience Years,CV,CV_lang,id,__index_level_0__
0,"13 years of exp || Solidity, C#, JavaScript ||...",Who am I:\r\n- 13 years of commercial experien...,I am interested in:\r\n- part-time engagement;...,Landed a role of Director of Blockchain Develo...,Lead,fluent,11.0,Landed a role of Director of Blockchain Develo...,en,50534b61-6826-52b1-9ac5-bfd2cfa348ec,24230
1,1c Developer,Worked on a mobile application for tracking trips,None,None,Other,intermediate,3.0,\nWorked on a mobile application for tracking ...,en,c2b9ea56-b5c8-50ec-a63b-053a5c8ff467,24231
2,1C developer,1 am an 1C developer. I deployed an 1C to typo...,None,None,Flutter,intermediate,11.0,\n1 am an 1C developer. I deployed an 1C to ty...,en,b3bfe3ed-ec25-56b2-8aaa-8c629120538e,24232
3,1C Developer,Perfect knowledge of 1C:Enterprise Platform.\...,None,None,Other,intermediate,11.0,\nPerfect knowledge of 1C:Enterprise Platform...,en,163fb9a3-3695-5dc3-b1d0-e0038ce4d0a5,24233
4,#1 Customer Support Specialist,"Tech addicted, experienced customer support wi...",In terms of the nature of the Customer Support...,"thousands of resolved problems, also, thousand...",Support,fluent,2.5,"thousands of resolved problems, also, thousand...",en,3f201183-db58-5333-9139-74c16059dc4a,24234


In [ ]:
candidate_profiles_df['Position'].value_counts().nlargest(10)

,count
Position,
QA Engineer,6908
Project Manager,6643
Front-end developer,5034
Junior QA Engineer,4940
Manual QA Engineer,3625
Java Developer,3607
UI/UX Designer,3569
Junior Front-end Developer,2946
Front-End Developer,2806


In [4]:
selected_position = 'Project Manager'
filtered_jobs_df = job_descriptions_df[job_descriptions_df['Position'] == (selected_position)]
filtered_jobs_df

,Position,Long Description,Company Name,Exp Years,Primary Keyword,English Level,Published,Long Description_lang,id,__index_level_0__
91899,Project Manager,"1,5+ years of experience in Project Management...",Master Of Code Global,2y,Project Manager,upper,2023-01-01T00:00:00+02:00,en,9c387041-9da8-5234-99f3-fbddc3329927,119360
91900,Project Manager,"1,5+ years of experience in ІТ Project Managem...",TechMagic,2y,Project Manager,upper,2020-09-01T00:00:00+03:00,en,f8a491dc-351e-571f-a413-350df6fb30b0,119361
91901,Project Manager,1) Defining of project goals and requirements\...,Luxoft,2y,Project Manager,upper,2021-04-01T00:00:00+03:00,en,ee93dfa5-d03e-5fd9-857c-fda797dd8688,119362
91902,Project Manager,1) Experience in IT (managing team of develope...,Prof-IT,2y,Project Manager,intermediate,2022-10-01T00:00:00+03:00,en,b170162f-dfdf-5fa0-b592-fb5e9c7a1c99,119363
91903,Project Manager,1.\tTeam Building and Management:\r\n●\tRecrui...,OBSGroup,2y,Project Manager,upper,2023-09-01T00:00:00+03:00,en,96fdcfd7-f857-57fe-a1d1-73ef8442f40a,119364
...,...,...,...,...,...,...,...,...,...,...
92960,Project Manager,Предлагаем\r\nWe offer:\r\n\r\n— Competitive c...,HotelFriend,1y,Project Manager,intermediate,2022-01-01T00:00:00+02:00,en,716b9784-39a4-549f-bb77-9b46b6eee982,120421
92961,Project Manager,"Продуктовая IT-компания, высоконагруженный про...",IT Rex,3y,Project Manager,upper,2020-07-01T00:00:00+03:00,en,4288f548-17f2-5d24-8865-393d330ec259,120422
92962,Project Manager,Проект: \r\nПідбирається на другому етапі спів...,IT-Rex,2y,Project Manager,upper,2022-06-01T00:00:00+03:00,en,8e20ac5c-c9b7-53f8-8d29-a6a28e277bd3,120423
92963,Project Manager,✅Требования\r\n3+ years of experience in manag...,Benefit - Recruiting Company,3y,Project Manager,upper,2021-04-01T00:00:00+03:00,en,40bc646f-658c-5ac9-94ac-b87bd0be28d6,120424


In [5]:
selected_position = 'Project Manager'
filtered_candidates_df = candidate_profiles_df[candidate_profiles_df['Position'] == ('Project Manager')]
filtered_candidates_df

,Position,Moreinfo,Looking For,Highlights,Primary Keyword,English Level,Experience Years,CV,CV_lang,id,__index_level_0__
130037,Project Manager,01.2020 - 04.2020\r\nProject manager/Account m...,Seeking for the company with cool people to co...,- Managing multiple projects at a time\r\n- Ex...,Project Manager,upper,2.0,- Managing multiple projects at a time\r\n- Ex...,en,08130374-27aa-5c92-954f-4e6b7f528fea,154267
130038,Project Manager,"02.2018 - 05.2020\r\n""Alfa-Bank""\r\nIT Project...",None,COURSES AND CERTIFICATES\r\n- PMP exam prepara...,Project Manager,upper,3.0,COURSES AND CERTIFICATES\r\n- PMP exam prepara...,en,7cae2ae8-951c-5873-870b-e72c8421a2d7,154268
130039,Project Manager,02/2020 – 06/2022 teaching activity at Departm...,None,I am very diversified and multitasking person....,Project Manager,fluent,2.0,I am very diversified and multitasking person....,en,157029ca-a74d-59a2-9576-a90c6224b2d2,154269
130040,Project Manager,03.2016 - 02.2019\r\nDevSup Studio\r\nMy first...,"At the moment, I am looking for a company with...",Grew own development studio from 5 to 30+ peop...,Project Manager,upper,6.0,Grew own development studio from 5 to 30+ peop...,en,a084fd34-6b89-5f05-801b-aa6fb21d846f,154270
130041,Project Manager,03.2017-10.2017 IT Pro...,"Development of modern applications, improvemen...",Managing multiple projects and deadlines,Project Manager,intermediate,1.0,Managing multiple projects and deadlines\n03.2...,en,a455e89b-d468-5ad6-8ece-baa1457b0745,154271
...,...,...,...,...,...,...,...,...,...,...,...
136675,Project Manager,-\tОбщий опыт работы в IT индустрии: 10 лет;\r...,I am interested in the project management role...,I was involved in the development of the large...,Project Manager,fluent,10.0,I was involved in the development of the large...,en,82c534a2-3123-5eee-917f-4024ffdd62aa,160905
136676,Project Manager,Опыт работы в сфере НЕ IT проектов - более 15 ...,To realize my skills and ambitions;\r\nChange ...,GLOBAL ABC. MARCH 2017 – APRIL 2018\r\nRESPONS...,Project Manager,intermediate,0.0,GLOBAL ABC. MARCH 2017 – APRIL 2018\r\nRESPONS...,en,b172848f-76bd-5add-8255-3783ff51fe4d,160906
136677,Project Manager,Сurrent position: Team Lead of Affiliate Teams...,None,None,Project Manager,intermediate,3.0,\nСurrent position: Team Lead of Affiliate Tea...,en,32e8adff-ad12-5c6b-8b60-f29e39e4abca,160907
136678,Project Manager,Сurrent tasks/Duties:\r\n - Addin...,None,Google certificates:\r\n- Сертификация Google ...,Project Manager,upper,5.0,Google certificates:\r\n- Сертификация Google ...,en,9b167250-aecf-5093-98f9-18dc9e40a508,160908


In [6]:
original_cv_df = filtered_candidates_df.copy()

In [ ]:
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create a text2text generation pipeline
local_llm = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=-1)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def rewrite_text_with_job(text, job_description):
    if pd.isna(text) or text.strip() == "" or pd.isna(job_description) or job_description.strip() == "":
        return text  # Return original if missing

    prompt = f"""
You are a professional resume editor.

Rewrite the following candidate's CV to better match the job description below.
Focus on emphasizing matching skills, experience, and qualifications.
Keep it realistic and professional. Avoid copying the job description directly.

Job Description:
{job_description}

Original CV:
{text}

Rewritten CV:
"""
    result = local_llm(prompt, max_length=1150, do_sample=False)[0]['generated_text']
    return result.strip()


def find_matching_job(candidate_position):
    matches = job_descriptions_df[job_descriptions_df['Position'].str.contains(candidate_position, case=False, na=False)]
    if not matches.empty:
        return matches.iloc[0]['Long Description']
    else:
        return None

In [7]:
full_df = original_cv_df.reset_index(drop=True)

job_description_text = filtered_jobs_df.iloc[25]['Long Description']

df1, df2, df3, df4, df5 = np.array_split(full_df, 5)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
df5[['CV']].to_csv("df5_cv.csv", index=False)
files.download("df5_cv.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
job_description_text

"About the project:\r\n\r\nOur customer is one of the leading ethereum miners in the world. We are going to work on performance improvements for better mining profit.\r\n\r\nWe're looking for someone with:\r\n\r\n• Experience in IT and development process;\r\n• Understanding of different Agile methodologies;\r\n• Knowledge of project management methodology and the process of software development;\r\n• Good written and verbal communication skills;\r\n• Willing to understand technical details of crypto project;\r\n• Speaking English level – Upper intermediate.\r\n\r\nWill be an advantage:\r\n\r\n• Experience in cryptocurrency;\r\n• Previous Experience in roles such as QA, BA, etc.\r\n\r\nYou will be responsible for:\r\n\r\n• Work with development team (6 people);\r\n• Update stakeholders on project status;\r\n• Write User Stories as a part of requirement gathering process;\r\n• Conduct regular reporting to top management;\r\n• Facilitate regular delivery process.\r\n\r\nWhat can we offer

In [9]:
with open("job_description.txt", "w") as file:
    file.write(job_description_text)

In [10]:
from google.colab import files
files.download("job_description.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# tqdm.pandas()  # Add progress bar
df1['Rewritten_CV'] = df1.progress_apply(lambda row: rewrite_text_with_job(row['CV'], job_description_text), axis=1)
df2['Rewritten_CV'] = df2.progress_apply(lambda row: rewrite_text_with_job(row['CV'], job_description_text), axis=1)
df3['Rewritten_CV'] = df3.progress_apply(lambda row: rewrite_text_with_job(row['CV'], job_description_text), axis=1)
df4['Rewritten_CV'] = df4.progress_apply(lambda row: rewrite_text_with_job(row['CV'], job_description_text), axis=1)
df5['Rewritten_CV'] = df5.progress_apply(lambda row: rewrite_text_with_job(row['CV'], job_description_text), axis=1)


# candidate_llm_df = full_df

# candidate_llm_df

  0%|          | 1/1329 [00:00<00:06, 194.23it/s]


NameError: name 'local_llm' is not defined

In [ ]:
# sampled_df = original_cv_df.sample(n=50, random_state=42).reset_index(drop=True)
# original_texts = sampled_df['CV'].fillna("").tolist()
# batch_size = 5
# rewritten_texts = []

# for i in tqdm(range(0, len(original_texts), batch_size)):
#     batch = original_texts[i:i+batch_size]
#     rewritten_batch = batch_candidate_llm_rewrite(batch)
#     rewritten_texts.extend(rewritten_batch)

# sampled_df = sampled_df.iloc[:len(rewritten_texts)].copy()
# sampled_df['Rewritten_CV'] = rewritten_texts

# candidate_llm_df = sampled_df

In [ ]:
# candidate_llm_df['Rewritten'] = candidate_llm_df['Highlights'].apply(candidate_llm_rewrite)
candidate_llm_df = sampled_df

HTTPError: 429 Client Error: Too Many Requests for url: https://api.groq.com/openai/v1/chat/completions